# Introduction
This project is purely for fun and for people who want to find good restaurants in their city like I am. 
So i will restrain city to chicago. 

Also, this project will explore both recommenderlab and Lightfm to compare their performances.

In [4]:
!pip install simplejson

  Using cached simplejson-3.17.5-cp38-cp38-macosx_10_9_x86_64.whl (74 kB)


In [5]:
## packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightfm import LightFM
from lightfm.evaluation import precision_at_k,auc_score
import scipy
import simplejson as json


In [11]:
## load data
cities = []
path = '/Users/vikaschaturvedi/Documents/T-Systems/POC/Recommender/Yelp-Recommender-master/data/yelp_dataset/business.json'


In [25]:
with open(path) as fin:
    i = 0 
    for line in fin:
        line_contents = json.loads(line)
        categories = line_contents['categories']
        city = line_contents['city']
        try:
            if 'Restaurants' in categories or 'Food' in categories:
                cities.append(city)
            #if 'Chicago' == city:
                #print(line_contents)
        
        except TypeError:
            print(line_contents)
            print(city)
            pass

{'business_id': 'EzXlnsWtBuRJw8avEMfBqw', 'name': 'AAA Cooper Transportation', 'address': '1215 Wells Branch Pkwy', 'city': 'Pflugerville', 'state': 'TX', 'postal_code': '78660', 'latitude': 30.4384664, 'longitude': -97.6612963, 'stars': 1.0, 'review_count': 5, 'is_open': 1, 'attributes': None, 'categories': None, 'hours': None}
Pflugerville
{'business_id': 'VD_mB3i4GG-Ra-sFqlxzeA', 'name': 'Parkway Cleansing Center', 'address': '695 Truman Hwy', 'city': 'Hyde Park', 'state': 'MA', 'postal_code': '02136', 'latitude': 42.2525487, 'longitude': -71.1188104, 'stars': 3.0, 'review_count': 6, 'is_open': 1, 'attributes': None, 'categories': None, 'hours': None}
Hyde Park
{'business_id': 'dp5RVEVtI8ZVW9xEZtcICw', 'name': 'Value Dry of Massachusetts', 'address': '41 Belvidere St', 'city': 'Boston', 'state': 'MA', 'postal_code': '02115', 'latitude': 42.3460843, 'longitude': -71.0858503, 'stars': 3.0, 'review_count': 9, 'is_open': 1, 'attributes': None, 'categories': None, 'hours': None}
Boston
{

In [26]:
a = pd.Series(cities)

In [27]:
b = pd.DataFrame({'city' : a})
c = b.groupby('city').size().reset_index()
c.columns =  ['city','count']

In [39]:
print('Total Cities Available:')
print(c['city'].nunique())

Total Cities Available:
499


In [33]:
c.sort_values(by = 'count',ascending = False).head(10)

,city,count
358,Portland,14378
443,Vancouver,12966
23,Austin,12688
16,Atlanta,10156
334,Orlando,9174
47,Boston,7020
96,Columbus,6312
370,Richmond,2252
68,Cambridge,1900
63,Burnaby,1838


Let's choose one `city` for our complete analysis. Chossing **Atlanta** 

In [34]:
# fix for Ausin right now
mydata = []
with open(path) as fin:
    i = 0 
    for line in fin:
        line_contents = json.loads(line)
        categories = line_contents['categories']
        city = line_contents['city']
        try:
            if ('Restaurants' in categories or 'Food' in categories) and 'Atlanta' == city:
                mydata.append(line_contents)
                
        except TypeError:
            #print(line_contents)
            print(city)

Pflugerville
Hyde Park
Boston
Portland
Austin
Peabody
Boston
Boston
Portland
Austin
Columbus
Vancouver
Stone Mountain
Portland
Roslindale
Decatur
Watertown
Winder
Columbus
Salem
Vancouver
Braintree
Columbus
Waltham
Portland
Randolph
Portland
Atlanta
Austin
Waltham
Austin
Salem
Middleton
Austin
Clackamas
Vancouver
Bedford
Malden
Austin
Brookline
Beaverton
Atlanta
Brookline
Quincy
Norwood
Columbus
Portland
Vancouver
Boston
Stoughton
Portland
Boulder
Lake Oswego
Boston
Austin
Austin
Portland
Revere
Danvers
Portland
Austin
Maitland
Dripping Springs
Boston
Burlington
Somerville
Orlando
Dripping Springs
Austin
Maitland
Avondale Estates
Austin
Decatur
Portland
Austin
Portland
Beaverton
Vancouver
Austin
Boston
Orlando
Austin
Grove City
Portland
Somerville
West Newton
Austin
Boston
Clackamas
Waltham
Dorchester
Oviedo
Woburn
Atlanta
Needham
Portland
Quincy
Portland
Pflugerville
Portland
Austin
Weymouth
Beaverton
Columbus
Clackamas
Waltham
Portland
Sorrento
Atlanta
Boring
Smyrna
Atlanta
Cocoa
Col

In [40]:
len(categories)

66

In [41]:
len(city)

17

In [42]:
mydata[1:10]

[{'business_id': 'Q2vefh0tGhtCGQDK1FI7cw',
  'name': 'Piece of Cake',
  'address': '3215 Roswell Rd NE',
  'city': 'Atlanta',
  'state': 'GA',
  'postal_code': '30305',
  'latitude': 33.8428749,
  'longitude': -84.3785385,
  'stars': 4.0,
  'review_count': 231,
  'is_open': 1,
  'attributes': {'BikeParking': 'True',
   'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
   'Caters': 'True',
   'RestaurantsPriceRange2': '2',
   'BusinessAcceptsCreditCards': 'True',
   'RestaurantsTakeOut': 'True',
   'RestaurantsDelivery': 'True',
   'HappyHour': 'False'},
  'categories': 'Restaurants, Patisserie/Cake Shop, Desserts, Food, Cupcakes, Bakeries',
  'hours': {'Monday': '0:0-0:0',
   'Tuesday': '10:0-17:0',
   'Wednesday': '10:0-17:0',
   'Thursday': '10:0-17:0',
   'Friday': '10:0-17:0',
   'Saturday': '10:0-17:0',
   'Sunday': '12:0-17:0'}},
 {'business_id': 'TbZDLpBOl-EbO2LfMySrEg',
  'name': 'First China Restaurant',
  'address': '529

In [43]:
column_names = mydata[1].keys()
for item in mydata:
    
    if len([x for x in item.keys() if x in column_names]) != len(item.keys()):
        print(item.keys())

In [44]:
data = []
for column in column_names:
    this_col = []
    for item in mydata:
        this_col.append(item[column])
    data.append(this_col)


In [45]:
np.array(data[12])

array(['Food, Restaurants, Cheesesteaks, Sandwiches, Juice Bars & Smoothies',
       'Restaurants, Patisserie/Cake Shop, Desserts, Food, Cupcakes, Bakeries',
       'Restaurants, Chinese', ..., 'Restaurants, Mexican, Tex-Mex',
       'American (New), Restaurants, Cajun/Creole',
       'Restaurants, Mexican, Latin American'], dtype='<U279')

In [46]:
column_names

dict_keys(['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours'])

In [50]:
first_cols = list(column_names)[0:10]
#first_cols.append(list(column_names)[12])
pd_data ={}
idx = 0
for column in first_cols:
    pd_data[column] = data[idx]
    idx+=1
business = pd.DataFrame(pd_data)
print('Business dataset size: ',business.shape)
business.head()

Business dataset size:  (5078, 10)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count
0,z8-_6l5EhX5NuPfWzJYQMA,Great Wraps,3393 Peachtree Rd NE,Atlanta,GA,30326,33.846335,-84.363578,3.0,11
1,Q2vefh0tGhtCGQDK1FI7cw,Piece of Cake,3215 Roswell Rd NE,Atlanta,GA,30305,33.842875,-84.378539,4.0,231
2,TbZDLpBOl-EbO2LfMySrEg,First China Restaurant,5295 Buford Hwy NE,Atlanta,GA,30340,33.896640,-84.280918,3.0,11
3,bP6goJODwRnM3AVy45Kn9w,Papi's Cuban & Caribbean Grill,216 Ponce De Leon Ave NE,Atlanta,GA,30308,33.772758,-84.380375,4.0,1001
4,yCeENvVAGQQ0PeeWU0Pl2g,Nature's Table Bistro,"6000 N Terminal Pkwy, Concourse E Centrepoint,...",Atlanta,GA,30320,33.640879,-84.425514,3.5,150


In [51]:
business['category'] = data[12]
print(business.shape)
business.head()

(5078, 11)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,category
0,z8-_6l5EhX5NuPfWzJYQMA,Great Wraps,3393 Peachtree Rd NE,Atlanta,GA,30326,33.846335,-84.363578,3.0,11,"Food, Restaurants, Cheesesteaks, Sandwiches, J..."
1,Q2vefh0tGhtCGQDK1FI7cw,Piece of Cake,3215 Roswell Rd NE,Atlanta,GA,30305,33.842875,-84.378539,4.0,231,"Restaurants, Patisserie/Cake Shop, Desserts, F..."
2,TbZDLpBOl-EbO2LfMySrEg,First China Restaurant,5295 Buford Hwy NE,Atlanta,GA,30340,33.896640,-84.280918,3.0,11,"Restaurants, Chinese"
3,bP6goJODwRnM3AVy45Kn9w,Papi's Cuban & Caribbean Grill,216 Ponce De Leon Ave NE,Atlanta,GA,30308,33.772758,-84.380375,4.0,1001,"Caribbean, Cuban, Restaurants"
4,yCeENvVAGQQ0PeeWU0Pl2g,Nature's Table Bistro,"6000 N Terminal Pkwy, Concourse E Centrepoint,...",Atlanta,GA,30320,33.640879,-84.425514,3.5,150,"Restaurants, Sandwiches, Fast Food, American (..."


In [14]:
# write csv
#business.to_csv('/Users/ivanchen/Desktop/MSIA/Studying_materials/yelp_dataset/business.csv')


In [135]:
# read csv
business = pd.read_csv('/Users/ivanchen/Desktop/MSIA/Studying_materials/yelp_dataset/business.csv')





In [52]:
#get all nested columns
def get_superset_of_column_names_from_file(data):
    """Read in the json dataset file and return the superset of column names."""
    column_names = set()
    
    for line_contents in data:
        column_names.update(
                set(get_column_names(line_contents))
                )
    return column_names

def get_column_names(line_contents):
    """Return a list of flattened key names given a dict.
    Example:
        line_contents = {
            'a': {
                'b': 2,
                'c': 3,
                },
        }
        will return: ['a.b', 'a.c']
    These will be the column names for the eventual csv file.
    """
    column_names = []
    if line_contents is None:
        return ''
    for k in line_contents.keys():
        column_names.append('{0}'.format(k)
        )
    
    return column_names


In [53]:
nested_columns = get_superset_of_column_names_from_file(data[11])

In [54]:
nested_columns

{'AcceptsInsurance',
 'AgesAllowed',
 'Alcohol',
 'Ambience',
 'BYOB',
 'BYOBCorkage',
 'BestNights',
 'BikeParking',
 'BusinessAcceptsBitcoin',
 'BusinessAcceptsCreditCards',
 'BusinessParking',
 'ByAppointmentOnly',
 'Caters',
 'CoatCheck',
 'Corkage',
 'DietaryRestrictions',
 'DogsAllowed',
 'DriveThru',
 'GoodForDancing',
 'GoodForKids',
 'GoodForMeal',
 'HappyHour',
 'HasTV',
 'Music',
 'NoiseLevel',
 'Open24Hours',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsCounterService',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTableService',
 'RestaurantsTakeOut',
 'Smoking',
 'WheelchairAccessible',
 'WiFi'}

In [55]:

# make all cols zeroes first
for col in nested_columns:
    business[col]= np.nan
#loop through each attribute
idx = 0
for attr in data[11]:
    print(idx)
    if attr is None:
        idx+=1
        continue
    for k,v in attr.items():
        #print('value: '+v)
        if'{' in v:
            continue
        business[k][idx] = v
    idx+=1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

/var/folders/_7/2tzln8f91p32zpbhqd24ypj80000gn/T/ipykernel_30458/2352094549.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  business[k][idx] = v
/Users/vikaschaturvedi/opt/anaconda3/envs/cpe-replacement/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)



355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604

Let's look at how many NA it has

In [56]:
a = pd.DataFrame(business.isna().sum())
a

,0
business_id,0
name,0
address,0
city,0
state,0
postal_code,0
latitude,0
longitude,0
stars,0
review_count,0


In [57]:
a[a.iloc[:,0] > 2500] # keep columns that has at least half non na values.

,0
RestaurantsCounterService,5076
DogsAllowed,3827
BusinessParking,4999
ByAppointmentOnly,4656
DriveThru,4563
BYOB,4770
GoodForDancing,4635
AgesAllowed,5073
Open24Hours,5074
DietaryRestrictions,5078


In [58]:
# get rid of mostly NA columns
col_del = list(a[a.iloc[:,0] >2500].index)# columns to keep
col_del

['RestaurantsCounterService',
 'DogsAllowed',
 'BusinessParking',
 'ByAppointmentOnly',
 'DriveThru',
 'BYOB',
 'GoodForDancing',
 'AgesAllowed',
 'Open24Hours',
 'DietaryRestrictions',
 'AcceptsInsurance',
 'BusinessAcceptsBitcoin',
 'Smoking',
 'GoodForMeal',
 'CoatCheck',
 'BYOBCorkage',
 'Corkage',
 'Ambience',
 'WheelchairAccessible',
 'BestNights',
 'HappyHour',
 'Music',
 'RestaurantsTableService']

In [59]:
business.drop(columns=col_del).isna().sum()

business_id                      0
name                             0
address                          0
city                             0
state                            0
postal_code                      0
latitude                         0
longitude                        0
stars                            0
review_count                     0
category                         0
WiFi                          1698
RestaurantsGoodForGroups      1519
HasTV                         1714
RestaurantsAttire             1736
NoiseLevel                    2121
OutdoorSeating                1214
RestaurantsDelivery            794
RestaurantsReservations       1429
Caters                        1886
Alcohol                       1633
GoodForKids                   1547
BikeParking                   1700
RestaurantsPriceRange2         615
RestaurantsTakeOut             543
BusinessAcceptsCreditCards     302
dtype: int64

In [60]:
business = business.drop(columns=col_del)

In [63]:
# write csv
business.to_csv('/Users/vikaschaturvedi/Documents/T-Systems/POC/Recommender/Yelp-Recommender-master/data/interim/business_atlanta.csv',index=False)

In [64]:
business_test = pd.read_csv('/Users/vikaschaturvedi/Documents/T-Systems/POC/Recommender/Yelp-Recommender-master/data/interim/business_atlanta.csv')
print(business.shape)
business_test.head()

(5078, 26)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,OutdoorSeating,RestaurantsDelivery,RestaurantsReservations,Caters,Alcohol,GoodForKids,BikeParking,RestaurantsPriceRange2,RestaurantsTakeOut,BusinessAcceptsCreditCards
0,z8-_6l5EhX5NuPfWzJYQMA,Great Wraps,3393 Peachtree Rd NE,Atlanta,GA,30326.0,33.846335,-84.363578,3.0,11,...,NaN,True,False,NaN,NaN,True,NaN,NaN,True,True
1,Q2vefh0tGhtCGQDK1FI7cw,Piece of Cake,3215 Roswell Rd NE,Atlanta,GA,30305.0,33.842875,-84.378539,4.0,231,...,NaN,True,NaN,True,NaN,NaN,True,2.0,True,True
2,TbZDLpBOl-EbO2LfMySrEg,First China Restaurant,5295 Buford Hwy NE,Atlanta,GA,30340.0,33.896640,-84.280918,3.0,11,...,False,False,NaN,NaN,u'beer_and_wine',True,NaN,2.0,True,True
3,bP6goJODwRnM3AVy45Kn9w,Papi's Cuban & Caribbean Grill,216 Ponce De Leon Ave NE,Atlanta,GA,30308.0,33.772758,-84.380375,4.0,1001,...,None,True,False,False,'full_bar',True,True,2.0,True,True
4,yCeENvVAGQQ0PeeWU0Pl2g,Nature's Table Bistro,"6000 N Terminal Pkwy, Concourse E Centrepoint,...",Atlanta,GA,30320.0,33.640879,-84.425514,3.5,150,...,False,False,False,False,u'none',True,False,2.0,True,True
